<a href="https://colab.research.google.com/github/shizukuyu/LLM/blob/master/match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# 假设已经加载了主表和参照表数据
main_table = pd.read_excel("task.xlsx")
reference_table = pd.read_excel("matrix.xlsx")

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
!pip install fuzzywuzzy


In [ ]:
columns_to_drop = ['操作人', '复核人', '触发零报', '具体冗余', '执行情况', '备注','Unnamed: 14']
main_table.drop(columns=columns_to_drop, inplace=True)
main_table.loc[main_table['专业'] == '强电', '专业'] = '电气'
main_table = main_table.fillna('')

In [ ]:
columns_to_keep = ['大项序号', '动作序号', '专业', '设备名称', '操作类别', '操作名称', '系统检查', '操作影响', '操作风险', '切入场景','PPM场景','操作定级','升/降级逻辑']
reference_table = reference_table.loc[:, columns_to_keep]
reference_table = reference_table.dropna(subset=['动作序号'])
reference_table['大项序号'] = reference_table['大项序号'].ffill()
reference_table = reference_table.fillna('')
reference_table

,大项序号,动作序号,专业,设备名称,操作类别,操作名称,系统检查,操作影响,操作风险,切入场景,PPM场景,操作定级,升/降级逻辑
1,1.0,1.0,电气,列头柜,参数调整,列头柜开关保护参数修改操作,BMS可以监控,直接影响末端,修改错误，列头柜无法正常工作,系统优化，区域要求、客户要求、风险整改,/,1级,无需升降级
2,1.0,2.0,电气,列头柜,参数调整,列头柜监测终端内参数调整,远程无法监控,不影响末端,,,,3级,
3,1.0,3.0,电气,列头柜,启停操作,列头柜总进线分合闸,BMS可以监控,直接影响末端,列头柜失电，末端业务单路掉电,变更、故障检修,/,1级,无业务机房降为3级
4,1.0,4.0,电气,列头柜,启停操作,列头柜空开合闸操作,BMS可以监控,间接影响末端,误触碰开关,上下电场景,/,2级,无需升降级
5,1.0,5.0,电气,列头柜,启停操作,列头柜空开分闸操作,BMS可以监控,直接影响末端,下错机柜，机柜单路掉电,上下电场景,/,1级,无需升降级
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,57.0,202.0,物业,工具,在线操作,工具检查,远程无法监控,不影响末端,无影响,PPM,工具、仪表检查,3级,无需升降级
203,58.0,203.0,物业,防雷设施,在线操作,防雷设施检查作业,远程无法监控,不影响末端,无影响,PPM,防雷设施检测,3级,无需升降级
204,59.0,204.0,物业,电梯,设备退出,电梯启停操作,远程无法监控,不影响末端,无影响,PPM,电梯例行保养\n电梯第三方年检,3级,涉及人员安全提级1级
205,60.0,205.0,物业,液压升降平台,在线操作,液压升降平台例行巡检,远程无法监控,不影响末端,无影响,PPM,液压升降平台例行巡检,3级,无需升降级


In [ ]:
print(main_table.columns)
print(reference_table.columns)


Index(['Date', '作业类型', '数据中心', '作业标题', '风险等级', '专业', '作业内容', '风险描述'], dtype='object')
Index(['大项序号', '动作序号', '专业', '设备名称', '操作类别', '操作名称', '系统检查', '操作影响', '操作风险',
       '切入场景', 'PPM场景', '操作定级', '升/降级逻辑'],
      dtype='object')


In [ ]:
main_table.columns = main_table.columns.str.strip()
reference_table.columns = reference_table.columns.str.strip()

In [ ]:
main_table['combined_text'] = main_table.apply(
    lambda row: f"{row['作业类型']} {row['作业标题']} {row['专业']} {row['作业内容']} {row['风险描述']}", axis=1)


# reference_table['combined_ref_text'] = reference_table.apply(
#     lambda row:
#     f"{row['动作序号']} {row['设备名称']} {row['专业']} {row['操作类别']} {row['操作名称']}{row['切入场景']}{row['PPM场景']}{row['操作定级']}", axis=1
# )


In [ ]:
from fuzzywuzzy import process

def fuzzy_match(row, ref_table, ref_column):
    # 在参考表的指定列中找到与作业内容最匹配的值
    result = process.extractOne(row['combined_text'], ref_table[ref_column], scorer=fuzz.partial_ratio)
    print(f"DEBUG: result={result}")  # 调试信息，查看结果
    if result and len(result) >= 3:
        match, score, index = result  # 解包匹配项、得分和索引
        if score > 0.0001:  # 调整阈值
            return match, score, index
        else:
            return None, score, None
    else:
        return None, 0, None


In [ ]:
# # 在设备名称、操作类别和操作名称列中进行模糊匹配
# main_table['匹配_设备名称'] = main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='设备名称', axis=1)
# main_table['匹配_操作类别'] = main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='操作类别', axis=1)
# main_table['匹配_操作名称'] = main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='操作名称', axis=1)
# # main_table['匹配_系统检查'] = main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='系统检查', axis=1)
# # main_table['匹配_操作风险'] = main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='操作风险', axis=1)
# main_table['匹配_切入场景'] = main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='切入场景', axis=1)
# main_table['匹配_PPM场景'] = main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='PPM场景', axis=1)
# # main_table['匹配_升/降级逻辑'] = main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='升/降级逻辑', axis=1)

In [ ]:
main_table['匹配_设备名称'], main_table['设备名称_得分'], main_table['设备名称_索引'] = zip(*main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='设备名称', axis=1))
main_table['匹配_操作类别'], main_table['操作类别_得分'], main_table['操作类别_索引'] = zip(*main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='操作类别', axis=1))
main_table['匹配_操作名称'], main_table['操作名称_得分'], main_table['操作名称_索引'] = zip(*main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='操作名称', axis=1))
main_table['匹配_切入场景'], main_table['切入场景_得分'], main_table['切入场景_索引'] = zip(*main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='切入场景', axis=1))
main_table['匹配_PPM场景'], main_table['PPM场景_得分'], main_table['PPM场景_索引'] = zip(*main_table.apply(fuzzy_match, ref_table=reference_table, ref_column='PPM场景', axis=1))


流式输出内容被截断，只能显示最后 5000 行内容。
DEBUG: result=('故障维修', 50, 13)
DEBUG: result=('PPM', 100, 6)
DEBUG: result=('二次施工、故障维修、变更', 50, 194)
DEBUG: result=('变更', 100, 60)
DEBUG: result=('变更', 100, 60)
DEBUG: result=('变更', 100, 60)
DEBUG: result=('PPM', 100, 6)
DEBUG: result=('PPM', 100, 6)
DEBUG: result=('其它', 50, 206)
DEBUG: result=('PPM', 100, 6)
DEBUG: result=('PPM', 100, 6)
DEBUG: result=('故障维修', 50, 13)
DEBUG: result=('故障维修', 50, 13)
DEBUG: result=('变更', 100, 60)
DEBUG: result=('PPM', 100, 6)
DEBUG: result=('PPM', 100, 6)
DEBUG: result=('故障维修、变更', 57, 33)
DEBUG: result=('PPM', 100, 6)
DEBUG: result=('变更、风险消除', 50, 162)
DEBUG: result=('上下水服务器', 50, 140)
DEBUG: result=('其它', 50, 206)
DEBUG: result=('其它', 50, 206)
DEBUG: result=('变更、风险消除', 50, 162)
DEBUG: result=('故障维修', 50, 13)
DEBUG: result=('故障维修', 50, 13)
DEBUG: result=('故障维修', 50, 13)
DEBUG: result=('变更', 100, 60)
DEBUG: result=('变更', 100, 60)
DEBUG: result=('变更', 100, 60)
DEBUG: result=('其它', 50, 206)
DEBUG: result=('其它', 50, 206)
DEBUG: re

In [ ]:
有效index_list = []
操作定级_list = []

# 遍历每一行的索引列表
for idx_list in main_table[['设备名称_索引', '操作类别_索引', '操作名称_索引', '切入场景_索引', 'PPM场景_索引']].values.tolist():
    # DEBUG: 查看当前行的索引列表
    print(f"DEBUG: Index list for this row: {idx_list}")

    # 去除列表中的 NaN 值，保留有效的索引
    valid_indices = [int(idx) for idx in idx_list if not pd.isna(idx)]

    if valid_indices:
        有效index_list.append(valid_indices)

        # 收集所有有效索引对应的“操作定级”，并检查索引是否在范围内
        操作定级_values = []
        for idx in valid_indices:
            if 0 <= idx < len(reference_table):
                操作定级_values.append(reference_table.iloc[idx]['操作定级'])
            else:
                print(f"WARNING: Index {idx} is out-of-bounds for reference_table with length {len(reference_table)}")

        if 操作定级_values:
            # 将所有操作定级合并为一个字符串，用逗号分隔
            combined_操作定级 = ', '.join(操作定级_values)
            操作定级_list.append(combined_操作定级)
        else:
            # 如果没有有效的操作定级，返回 None
            操作定级_list.append(None)
    else:
        # 如果没有有效的索引，返回 None
        有效index_list.append(None)
        操作定级_list.append(None)

# 将 "有效index" 和 "操作定级" 列添加到 main_table 中
main_table['有效index'] = pd.Series(有效index_list, index=main_table.index)
main_table['操作定级'] = pd.Series(操作定级_list, index=main_table.index)


DEBUG: Index list for this row: [204.0, 206.0, 152.0, 206.0, 8.0]
DEBUG: Index list for this row: [62.0, 1.0, 195.0, 194.0, 202.0]
DEBUG: Index list for this row: [202.0, 3.0, 53.0, 194.0, 202.0]
DEBUG: Index list for this row: [83.0, 12.0, 195.0, 6.0, 91.0]
DEBUG: Index list for this row: [184.0, 147.0, 194.0, 147.0, 180.0]
DEBUG: Index list for this row: [183.0, 147.0, 194.0, 147.0, 164.0]
DEBUG: Index list for this row: [123.0, 1.0, 166.0, 60.0, 167.0]
DEBUG: Index list for this row: [47.0, 3.0, 199.0, 4.0, 8.0]
DEBUG: Index list for this row: [156.0, 14.0, 161.0, 60.0, 164.0]
DEBUG: Index list for this row: [202.0, nan, 195.0, 194.0, 202.0]
DEBUG: Index list for this row: [177.0, 12.0, 32.0, 60.0, 19.0]
DEBUG: Index list for this row: [1.0, 41.0, 19.0, 4.0, 8.0]
DEBUG: Index list for this row: [55.0, 41.0, 55.0, 60.0, 174.0]
DEBUG: Index list for this row: [204.0, 3.0, 195.0, 6.0, 115.0]
DEBUG: Index list for this row: [83.0, 3.0, 116.0, 60.0, 91.0]
DEBUG: Index list for this row: 

In [ ]:
main_table.to_excel('match-result.xlsx', index=False)

In [ ]:
print(操作定级_list)

['3级, 3级, 2级', '2级, 3级, 1级, 1级, 3级', '3级, 2级, 1级, 1级, 3级', '3级, 2级, 1级, 2级, 3级', '2级, 3级, 1级, 3级, 3级', '2级, 3级, 1级, 3级, 3级', '2级, 3级, 3级, 1级, 2级', '2级, 2级, 3级, 1级, 2级', '3级, 2级, 3级, 1级, 3级', '3级, 1级, 1级, 3级', '3级, 2级, 1级, 1级, 2级', '3级, 3级, 2级, 1级, 2级', '3级, 3级, 3级, 1级, 3级', '3级, 2级, 1级, 2级, 2级', '3级, 2级, 2级, 1级, 3级', '1级, 3级, 3级, 1级, 3级', '3级, 2级, 3级, 1级, 2级', '2级, 2级, 2级, 1级, 2级', '3级, 2级, 3级, 1级, 2级', '2级, 2级, 3级, 1级, 2级', '3级, 3级, 3级, 1级, 2级', '2级, 3级, 2级, 1级, 2级', '1级, 3级, 2级, 3级, 2级', '2级, 2级, 2级, 2级, 1级', '3级, 2级, 3级, 1级, 2级', '3级, 2级, 1级, 1级, 2级', '2级, 3级, 2级, 3级, 2级', '3级, 2级, 2级, 1级, 2级', '2级, 2级, 3级, 1级, 2级', '3级, 2级, 1级, 2级', '2级, 2级, 1级, 1级, 2级', '3级, 2级, 3级, 1级, 3级', '3级, 2级, 3级, 1级, 3级', '3级, 3级, 2级, 1级, 1级', '3级, 3级, 3级, 2级, 3级', '3级, 3级, 2级, 2级, 1级', '2级, 2级, 1级, 1级, 1级', '2级, 2级, 3级, 1级, 2级', '3级, 2级, 2级, 3级', '2级, 2级, 3级, 1级, 2级', '2级, 2级, 2级, 1级, 2级', '2级, 2级, 3级, 1级, 2级', '3级, 2级, 1级, 1级, 2级', '2级, 2级, 1级, 1级, 3级', '2级, 2级, 1级, 1级, 3级', '2级, 2级, 1级, 1级, 3级', '3级, 2级

In [ ]:
reference_table['combined_ref_text'] = reference_table.apply(
    lambda row:
    f"{row['动作序号']} {row['设备名称']} {row['专业']} {row['操作类别']} {row['操作名称']}{row['切入场景']}{row['PPM场景']}{row['操作定级']}", axis=1
)

In [ ]:
#这种是事件表合并内容和矩阵表合并内容进行的全局匹配

from fuzzywuzzy import process, fuzz

def fuzzy_match(row, ref_table):
    # 在 reference_table 的 combined_ref_text 列中找到最匹配的值
    result = process.extractOne(row['combined_text'], ref_table['combined_ref_text'], scorer=fuzz.partial_ratio)
    print(f"DEBUG: result={result}")  # 调试信息，查看结果
    if result and len(result) >= 3:
        match, score, index = result  # 解包匹配项、得分和索引
        if score > 70:  # 调整阈值
            return match, score, index
        else:
            return None, score, None
    else:
        return None, 0, None


In [ ]:
results = main_table.apply(fuzzy_match, ref_table=reference_table, axis=1)

# # 根据匹配索引查找 "操作定级"
# main_table['操作定级'] = main_table['匹配索引'].apply(lambda x: reference_table.iloc[x]['操作定级'] if pd.notna(x) else None)

DEBUG: result=('42.0 HVDC 电气 设备检修 在线更换模块故障维修/3级', 35, 42)
DEBUG: result=('65.0 精密空调 暖通 启停操作 精密空调开关机故障维修，单台精密空调维保、PPM行间/列间空调（半年度）维护保养\n风冷精密空调（季度）维护保养\n水冷精密空调（季度）维护保养\nVRV空调（半年度）维护保养2级', 29, 65)
DEBUG: result=('200.0 药剂 物业 在线操作 园区消杀PPM有害生物防治3级', 25, 200)
DEBUG: result=('88.0 冷却塔 暖通 在线操作 冷却塔目视巡检PPM/3级', 43, 88)
DEBUG: result=('51.0 BNM 电气 在线操作 在线更换风机故障维修/3级', 27, 51)
DEBUG: result=('196.0 EHS EHS 启停操作 吊装作业二次施工、故障维修、变更/1级', 29, 196)
DEBUG: result=('166.0 门禁系统 弱电 启停操作 更换门禁模块故障维修/2级', 22, 166)
DEBUG: result=('5.0 列头柜 电气 启停操作 列头柜空开分闸操作上下电场景/1级', 36, 5)
DEBUG: result=('162.0 BA系统 弱电 软件升级 BA系统升级变更、风险消除/3级', 43, 162)
DEBUG: result=('194.0 EHS EHS 启停操作 动火作业二次施工、故障维修、变更/1级', 21, 194)
DEBUG: result=('5.0 列头柜 电气 启停操作 列头柜空开分闸操作上下电场景/1级', 27, 5)
DEBUG: result=('3.0 列头柜 电气 启停操作 列头柜总进线分合闸变更、故障检修/1级', 29, 3)
DEBUG: result=('51.0 BNM 电气 在线操作 在线更换风机故障维修/3级', 30, 51)
DEBUG: result=('13.0 柴油发电机 电气 设备检修 柴发PLC退出故障维修/2级', 30, 13)
DEBUG: result=('116.0 冷源系统 暖通 状态调整 冷源轮值切换设备状态变更/2级', 35, 116)
DEBUG: result=('55.0

TypeError: Cannot index by location index with a non-integer key

In [ ]:
# 将结果拆分为三列：匹配内容、匹配得分和匹配索引
main_table[['匹配内容', '匹配得分', '匹配索引']] = pd.DataFrame(results.tolist(), index=main_table.index)

NameError: name 'table' is not defined

In [ ]:
main_table['匹配索引'] = pd.to_numeric(main_table['匹配索引'], errors='coerce')  # 将非数字值转换为 NaN


In [ ]:
main_table['操作定级'] = main_table['匹配索引'].apply(
    lambda x: reference_table.iloc[int(x)]['操作定级'] if pd.notna(x) else None
)
print(main_table['操作定级']
      )

0       None
1       None
2       None
3       None
4       None
        ... 
3419    None
3420    None
3421    None
3422    None
3423    None
Name: 操作定级, Length: 3424, dtype: object


In [ ]:
main_table.to_excel('all-match.xlsx', index=False)

In [ ]:
# 合并匹配到的结果和 reference_table 中的原始数据
result_table = main_table.merge(reference_table, left_on='匹配结果', right_on='combined_ref_text', how='left')

# # 保存结果
# result_table.to_excel('result.xlsx', index=False)
main_table.to_excel('all-match.xlsx', index=False)


In [ ]:
df = pd.read_csv('result_table.csv')

# 将DataFrame保存为XLSX文件
df.to_excel('result.xlsx', index=False)

# 新段落